<a href="https://colab.research.google.com/github/tcardlab/optimus_bind_sample/blob/develop/notebooks/5_0_TJC_Check_Existence_Of_AAs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install biopython
import pandas as pd
import numpy as np
import re
from Bio.PDB import *

In [0]:
csv_link = 'https://life.bsc.es/pid/skempi2/database/download/skempi_v2.csv'
!wget $csv_link -O skempi_v2.0.csv #-O to rename
skempi = pd.read_csv('skempi_v2.0.csv', sep=';')

#Download SKEMPI PDBs
pdb_link = 'https://life.bsc.es/pid/skempi2/database/download/SKEMPI2_PDBs.tgz'
!wget $pdb_link
!tar -xvzf SKEMPI2_PDBs.tgz

#Final Comparison

##Functional

Sang Young Noh of the Optimus Bind Project used an alternative method, using a lookup dict to match AA format rather than converting to sequence. 

It is a bit disconcerting that PPBuilder does not operate as intended. 

This may point to underlying issues in the PDB's or bug. Both worth further investigation

In [27]:
from Bio.PDB.Polypeptide import aa1
from Bio.PDB.Polypeptide import aa3

# AA lookup Dict
AA1to3 = dict(zip(aa1, aa3))

#Only runs against problem PDBs for efficiency
errdct = {
'2BTF': 4, 
'1DAN': 3, 
'3BTE': 1, 
'3BTG': 1, 
'3BTH': 1, 
'3BTW': 1, 
'3D3V': 6, 
'4NKQ': 47,
'3R9A': 0
}

#init:
err = dict()
unique_pdb = set()
parser = PDBParser(QUIET=True)  # PERMISSIVE=1
pattern = '|'.join(errdct.keys())
ln = len(skempi)#.shape[0]
skempi2 = skempi[skempi['#Pdb'].str.contains(pattern)]
#print(skempi)


#test:
for index, row in skempi2.iterrows():
  # Import PDB
  pdb=row['#Pdb'].split('_')[0]
  structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
  unique_pdb.add(pdb)
  
  #Progress
  percent = 100*index/ln if index > 0 else 0
  if index%100==0: 
    print(f'{int(percent)}%', f'err:{len(err)} total:{len(unique_pdb)}')
    
  # Parse PDB to residues list. 
  # Unnecessary to convert the whole chain, but was important for 
  # debugging PPbuilder, so I left it for comparison
  Chain_seq = dict()
  for chain in structure.get_chains():
    Chain_seq[chain.id] = [c.resname for c in chain.get_residues()]
    
  # Mutation Comparator
  for mut in row['Mutation(s)_cleaned'].split(','):  # initAA/Chain/Index/mutAA
    initAA, chain, loc, mutAA = re.findall('(\d+|.)', mut)
    try:
      if AA1to3[initAA] != Chain_seq[chain][int(loc)-1]:  # if initAA != index of given chain
        err[pdb] = err[pdb]+1 if pdb in err.keys() else 1  # cant guarantee pdb exists, so no +=
        print(mut[0], Chain_seq[chain][int(loc)-2:int(loc)+3], mut, row['#Pdb'])
        print(Chain_seq)
    except:
      print(index, pdb, mut)
  #if index==500:
  #  break

print(err)

29% err:0 total:2
43% err:0 total:2
88% err:0 total:9
{}


##Broken PPBuilder

There was no "to_one_letter_code" for a structure's AAs, so I figured PPBuilder would be most efficient. Unfortunately its a bit finicky. PDBs arent the most reliable or cleanest files, however its is strange that Structure[0][chain].get_residues() reads correctly yet PPBuilder cuts of some sequences. 

However its still possible I am overlooking an error in my own code, its hard to say ATM...

In [28]:
'''Mutations_cleaned V2'''
#import warnings
#warnings.filterwarnings("ignore")

#init:
err = dict()
unique_pdb = set()
parser = PDBParser(QUIET=True)  # PERMISSIVE=1
ppb = Polypeptide.PPBuilder(radius=1000)

#test:
for index, row in skempi.iterrows():
  # Import PDB
  pdb=row['#Pdb'].split('_')[0]
  structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
  unique_pdb.add(pdb)
  
  #Progress
  percent = 100*index/ln if index > 0 else 0
  if index%100==0: 
    print(f'{int(percent)}%', f'err:{len(err)} total:{len(unique_pdb)}')
    
  # Parse PDB to sequences
  chain_ids = [c.id for c in structure.get_chains()]
  chain_pp = [c.get_sequence() for c in ppb.build_peptides(structure)]
  Chain_seq = dict(zip(chain_ids, chain_pp))
  
  # Mutation Comparator
  for mut in row['Mutation(s)_cleaned'].split(','):  # initAA/Chain/Index/mutAA
    initAA, chain, loc, mutAA = re.findall('(\d+|.)', mut)
    try:
      if initAA != Chain_seq[chain][int(loc)-1]: # failed to locate mutation
        err[pdb] = err[pdb]+1 if pdb in err.keys() else 1
        print(mut[0], Chain_seq[chain][int(loc)-2:int(loc)+3], mut, row['#Pdb'])
        print(Chain_seq)
    except: # probably failed to index (out of range). PPBuilder cuts off... 
      print(index, pdb, mut)
  #if index==100:
  #  break

print(err)

0% err:0 total:1
1% err:0 total:21
2% err:0 total:27
4% err:0 total:40
5% err:0 total:49
7% err:0 total:56
8% err:0 total:68
F AMYVA FP59A 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}
V MYVAI VP60E 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}
G ILTER GP120F 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}
K GYSFT KP125A 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}
9% err:1 total:82
11% err:1 total

#**Index descriptions**

Source – https://life.bsc.es/pid/skempi2/info/faq_and_help

---



0.   <strong>'#Pdb' – </strong>
The PDB entry for the complex, followed by the chain identifiers for the two subunits. The first chain(s) correspond to protein 1 (column 10) and the second chain(s) correspond to protein 2 (column 11). Following this link will lead you to the relevant page in the protein databank.

0.   <strong>'Mutation(s)_PDB' – </strong>
The mutation(s) corresponding to the residue numbering found in the protein databank. The first character is the one letter amino acid code for the original residue, the second character is the chain identifier, the third to penultimate characters indicate the residue number, followed by the residue insertion code where applicable, and the final character indicates the mutant amino acid. Where multiple mutations are present, they are separated by commas.

0.  <strong>'Mutation(s)_cleaned' – </strong>
The mutation(s) corresponding to the residue numbering in the 'cleaned' pdb files, in the same format as for column 2.

0.   <strong>'iMutation_Location(s)' – </strong>
The locations of the mutations(s) in or away from the binding site, as defined in "A simple definition of structural regions in proteins and its use in analyzing interface evolution", ED Levy, J Mol Biol. 2010, 403(4):660-70.

0.   <strong>'Hold_out_type' – </strong>
Some of the complexes are classified as protease-inhibitor (Pr/PI), antibody-antigen (AB/AG) or pMHC-TCR (TCR/pMHC). This classification was introduced to aid in the cross-validation of empirical models trained using the data in the SKEMPI database, so that proteins of a similar type can be simultaneously held out during a cross-validation.

0.   <strong>'Hold_out_proteins' – </strong>
This column contains the PDB identifiers (in column 1) and/or hold-out types (column 5) for all the protein complexes which may be excluded from the training when cross-validating an empirical model trained on this data, so as to avoid contaminating the training set with information pertaining to the binding site being evaluated.

0.   <strong>'Affinity_mut (M)' – </strong>
The affinity of the mutant form (M).

0.   <strong>'Affinity_mut_parsed' – </strong>
The affinity of the mutant form (M).

0.   <strong>'Affinity_wt (M)' – </strong>
The affinity of the wild-type form, or form in the PDB structure (M).

0.    <strong>'Affinity_wt_parsed' – </strong>
The affinity of the wild-type form, or form in the PDB structure (M).

0.    <strong>'Reference' – </strong>
The reference for the affinities, as well as any further kinetic or thermodynamic information. Where available, the pubmed ID is given with a link to the relevant entry in pubmed, otherwise the whole reference is given.

0.    <strong>'Protein 1' – </strong>
This is the name of the protein which corresponds to the first chain(s) given in column 1.

0.   <strong>'Protein 2' – </strong>
This is the name of the protein which corresponds to the second chain(s) given in column 1.

0.   <strong>'Temperature' – </strong>
The temperature at which the experiment was performed.

0.   <strong>'kon_mut (M^(-1)s^(-1))' – </strong>
The association rate for the mutant protein, where available (M^(-1)s^(-1)).

0.   <strong>'kon_mut_parsed' – </strong>
The association rate for the mutant protein, where available (M^(-1)s^(-1)).

0.   <strong>'kon_wt (M^(-1)s^(-1))' – </strong>
The association rate for the wild-type protein or protein in the crystal structure, where available (M^(-1)S^(-1)).

0.   <strong>'kon_wt_parsed' – </strong>
The association rate for the wild-type protein or protein in the crystal structure, where available (M^(-1)S^(-1)).

0.   <strong>'koff_mut (s^(-1))' – </strong>
The dissociation rate for the mutant protein, where available (s^(-1)).

0.   <strong>'koff_mut_parsed' – </strong>
The dissociation rate for the mutant protein, where available (s^(-1)).

0.   <strong>'koff_wt (s^(-1))' – </strong>
The dissociation rate for the wild-type protein or protein in the crystal structure, where available (s^(-1)).

0.   <strong>'koff_wt_parsed' – </strong>
The dissociation rate for the wild-type protein or protein in the crystal structure, where available (s^(-1)).

0.   <strong>'dH_mut (kcal mol^(-1))' – </strong>
The enthalpy of association for the mutant protein, where available (kcal mol^(-1)).

0.   <strong>'dH_wt (kcal mol^(-1))' – </strong>
The enthalpy of association for the wild-type protein or protein in the crystal structure, where available (kcal mol^(-1)).

0.   <strong>'dS_mut (cal mol^(-1) K^(-1))' – </strong>
The entropy of association for the mutant protein, where available (cal mol^(-1) K^(-1)).

0.   <strong>'dS_wt (cal mol^(-1) K^(-1))' – </strong>
The entropy of association for the wild-type protein or protein in the crystal structure, where available (cal mol^(-1) K^(-1)).

0.   <strong>'Notes' – </strong>
Notes regarding the entry.

0.   <strong>'Method' – </strong>
The experimental method used to measure the affinities.

0.   <strong>'SKEMPI version' – </strong>
The SKEMPI version number.

#Tests

##Local index

looks like a couple of errs...

In [0]:
'''
Mutation(s)_PDB
should be used with fastA
so this should fail as pdb should be cleaned
'''
#import warnings
#warnings.filterwarnings("ignore")

#init:
err = dict()
unique_pdb = set()
parser = PDBParser(PERMISSIVE=0)
ppb = Polypeptide.PPBuilder()

#test:
for index, row in skempi.iterrows():
  # Import PDB
  pdb=row['#Pdb'].split('_')[0]
  structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
  unique_pdb.add(pdb)
  
  #Progress
  if index%100==0: print(f'err:{len(err)} total:{len(unique_pdb)}')
    
  # Parse PDB to sequences
  chain_ids = [c.id for c in structure.get_chains()]
  chain_pp = [c.get_sequence() for c in ppb.build_peptides(structure)]
  Chain_seq = dict(zip(chain_ids, chain_pp))
  for mut in row['Mutation(s)_PDB'].split(','):  # initAA/Chain/Index/mutAA
    initAA, chain, loc, mutAA = re.findall('(\d+|.)', mut)
    if initAA != Chain_seq[chain][int(loc)-1]: #if initAA != index of given chain
      err[pdb] = err[pdb]+1 if pdb in err.keys() else 1
      print(mut[0], Chain_seq[chain][int(loc)-2:int(loc)+3], mut, row['#Pdb'])
      #print(Chain_seq)
  if index==100:
    break

print(err)

In [16]:
'''Mutations_cleaned'''
#import warnings
#warnings.filterwarnings("ignore")

#init:
err = dict()
unique_pdb = set()
parser = PDBParser()
ppb = Polypeptide.PPBuilder()

#test:
for index, row in skempi.iterrows():
  # Import PDB
  pdb=row['#Pdb'].split('_')[0]
  structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
  unique_pdb.add(pdb)
  
  #Progress
  if index%100==0: print(f'err:{len(err)} total:{len(unique_pdb)}')
    
  # Parse PDB to sequences
  chain_ids = [c.id for c in structure.get_chains()]
  chain_pp = [c.get_sequence() for c in ppb.build_peptides(structure)]
  Chain_seq = dict(zip(chain_ids, chain_pp))
  for mut in row['Mutation(s)_cleaned'].split(','):  # initAA/Chain/Index/mutAA
    initAA, chain, loc, mutAA = re.findall('(\d+|.)', mut)
    if initAA != Chain_seq[chain][int(loc)-1]: #if initAA != index of given chain
      err[pdb] = err[pdb]+1 if pdb in err.keys() else 1
      print(mut[0], Chain_seq[chain][int(loc)-2:int(loc)+3], mut, row['#Pdb'])
      print(Chain_seq)
  if index==100:
    break

print(err)

F AMYVA FP59A 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}
V MYVAI VP60E 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}
G ILTER GP120F 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}
K GYSFT KP125A 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}


IndexError: ignored

In [0]:
'''
After seeing 1BRS pdb coverage on rcsb
(https://www.rcsb.org/pdb/explore/remediatedSequence.do?structureId=1BRS)
I realized the PDB sequence didnt capture all of the FASTA data.

Thus, if mutations happen after gaps in the sequence the index will be off.
This is where I relized the importance of the mapping files.

But it turns out I was incorrect... Biopython autosplits chains inf there is a 
break int he protein sequence. currently looking for an override... 
'''

#import warnings
#warnings.filterwarnings("ignore")

#init:
err = dict()
unique_pdb = set()
parser = PDBParser()
ppb = Polypeptide.PPBuilder()
ln = skempi.shape[0]

#test:
for index, row in skempi.iterrows():
  # Import PDB
  pdb=row['#Pdb'].split('_')[0]
  structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
  unique_pdb.add(pdb)
  
  #Progress
  percent = 100*ln/index if index > 0 else 0
  if index%500==0: 
    print(int(percent), f'err:{len(err)} total:{len(unique_pdb)}')
    
  # Parse PDB to sequences
  chain_ids = [c.id for c in structure.get_chains()]
  chain_pp = [c.get_sequence() for c in ppb.build_peptides(structure)]
  Chain_seq = dict(zip(chain_ids, chain_pp))
  for mut in row['Mutation(s)_cleaned'].split(','):  # initAA/Chain/Index/mutAA
    initAA, chain, loc, mutAA = re.findall('(\d+|.)', mut)
    try:
      if initAA != Chain_seq[chain][int(loc)-1]: #if initAA != index of given chain
        err[pdb] = err[pdb]+1 if pdb in err.keys() else 1
        print(mut[0], Chain_seq[chain][int(loc)-2:int(loc)+3], mut, row['#Pdb'])
        print(Chain_seq)
    except:
      err[pdb] = err[pdb]+1 if pdb in err.keys() else 1
      print(mut[0], Chain_seq[chain][int(loc)-2:int(loc)+3], mut, row['#Pdb'])
      print('except: ', Chain_seq)

print(err)

In [0]:
names=["Residue", "Chain", "FASTA_index", "PDB_index"]
data = pd.read_csv('PDBs/1BRS.mapping', sep=r"\s*", header=None, names=names)
data.loc[data['Chain'] == 'D']
data.set_index('FASTA_index')

In [0]:
def mapped_index(pdb, chain, index, basis='FASTA_index'):
  names = ["Residue", "Chain", "FASTA_index", "PDB_index"]
  map_data = pd.read_csv(f'PDBs/{pdb}.mapping', sep=r"\s*", 
                         header=None, names=names)
  map_data = map_data.loc[data['Chain'] == chain]
  map_data = map_data.set_index(basis)
  output = names[-2:][basis=='FASTA_index']
  return map_data[output][index]

# Given FASTA get PDB index
print(mapped_index('1BRS', 'D', 66, basis='FASTA_index'))
# Given PDB index get FASTA
print(mapped_index('1BRS', 'D', 64, basis='PDB_index'))

64
66


In [0]:
'''Mapped index'''
#import warnings
#warnings.filterwarnings("ignore")

def mapped_index(pdb, chain, index, basis='FASTA_index'):
  names = ["Residue", "Chain", "FASTA_index", "PDB_index"]
  map_data = pd.read_csv(f'PDBs/{pdb}.mapping', sep=r"\s*", 
                         header=None, names=names)
  map_data = map_data.loc[map_data['Chain'] == chain]
  map_data = map_data.set_index(basis)
  output = names[-2:][basis=='FASTA_index']
  return map_data[output][index]

#init:
err = dict()
unique_pdb = set()
parser = PDBParser()
ppb = Polypeptide.PPBuilder()

#test:
for index, row in skempi.iterrows():
  # Import PDB
  pdb=row['#Pdb'].split('_')[0]
  structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
  unique_pdb.add(pdb)
  
  #Progress
  percent = 100*index/ln if index > 0 else 0
  if index%100==0: 
    print(int(percent), f'err:{len(err)} total:{len(unique_pdb)}')
    
  # Parse PDB to sequences
  chain_ids = [c.id for c in structure.get_chains()]
  chain_pp = [c.get_sequence() for c in ppb.build_peptides(structure)]
  Chain_seq = dict(zip(chain_ids, chain_pp))
  for mut in row['Mutation(s)_PDB'].split(','):  # initAA/Chain/Index/mutAA
    try:
      initAA, chain, loc, mutAA = re.findall('(\d+|.)', mut)
      i = mapped_index(pdb, chain, int(loc)) - 1
    except:
      print(index)
    try:
      if initAA != Chain_seq[chain][i]: #if initAA != index of given chain
        err[pdb] = err[pdb]+1 if pdb in err.keys() else 1
        print(mut[0], Chain_seq[chain][i-2:i+3], mut, loc, i)
    except:
      print(index, pdb, mut, i, Chain_seq[chain])
  #if index==100:
  #  break

print(err)

0 err:0 total:1
L WVVTA LI45G 45 37
L WVVTA LI45S 45 37
L WVVTA LI45P 45 37
L WVVTA LI45I 45 37
L WVVTA LI45D 45 37
L WVVTA LI45E 45 37
1 err:1 total:21
116 1BRS ED76A 73 KKAVINGEQIRSISDLHQTLKKELALPEYYGENLDALWDALTGWVEYPLVLEWRQFEQSKQLT
117 1BRS ED80A 77 KKAVINGEQIRSISDLHQTLKKELALPEYYGENLDALWDALTGWVEYPLVLEWRQFEQSKQLT
123 1BRS ED76A 73 KKAVINGEQIRSISDLHQTLKKELALPEYYGENLDALWDALTGWVEYPLVLEWRQFEQSKQLT
124 1BRS ED80A 77 KKAVINGEQIRSISDLHQTLKKELALPEYYGENLDALWDALTGWVEYPLVLEWRQFEQSKQLT
129 1BRS ED76A 73 KKAVINGEQIRSISDLHQTLKKELALPEYYGENLDALWDALTGWVEYPLVLEWRQFEQSKQLT
130 1BRS ED80A 77 KKAVINGEQIRSISDLHQTLKKELALPEYYGENLDALWDALTGWVEYPLVLEWRQFEQSKQLT
136 1BRS ED76A 73 KKAVINGEQIRSISDLHQTLKKELALPEYYGENLDALWDALTGWVEYPLVLEWRQFEQSKQLT
137 1BRS ED80A 77 KKAVINGEQIRSISDLHQTLKKELALPEYYGENLDALWDALTGWVEYPLVLEWRQFEQSKQLT
142 1BRS ED76A 73 KKAVINGEQIRSISDLHQTLKKELALPEYYGENLDALWDALTGWVEYPLVLEWRQFEQSKQLT
143 1BRS ED80A 77 KKAVINGEQIRSISDLHQTLKKELALPEYYGENLDALWDALTGWVEYPLVLEWRQFEQSKQLT
149 1BRS ED76A 73 KKAVINGEQ

KeyboardInterrupt: ignored

In [0]:
'''Mutations_cleaned V2'''
#import warnings
#warnings.filterwarnings("ignore")

#init:
err = dict()
unique_pdb = set()
parser = PDBParser(QUIET=True)  # PERMISSIVE=1
ppb = Polypeptide.PPBuilder(radius=1000)

#test:
for index, row in skempi.iterrows():
  # Import PDB
  pdb=row['#Pdb'].split('_')[0]
  structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
  unique_pdb.add(pdb)
  
  #Progress
  percent = 100*index/ln if index > 0 else 0
  if index%100==0: 
    print(f'{int(percent)}%', f'err:{len(err)} total:{len(unique_pdb)}')
    
  # Parse PDB to sequences
  chain_ids = [c.id for c in structure.get_chains()]
  chain_pp = [c.get_sequence() for c in ppb.build_peptides(structure)]
  Chain_seq = dict(zip(chain_ids, chain_pp))
  for mut in row['Mutation(s)_cleaned'].split(','):  # initAA/Chain/Index/mutAA
    initAA, chain, loc, mutAA = re.findall('(\d+|.)', mut)
    try:
      if initAA != Chain_seq[chain][int(loc)-1]: #if initAA != index of given chain
        err[pdb] = err[pdb]+1 if pdb in err.keys() else 1
        print(mut[0], Chain_seq[chain][int(loc)-2:int(loc)+3], mut, row['#Pdb'])
        print(Chain_seq)
    except:
      print(index, pdb, mut)
  #if index==100:
  #  break

print(err)

##Test global index

good, its def not this

In [0]:
#import warnings
#warnings.filterwarnings("ignore")

#init:
err = dict()
unique_pdb = set()
parser = PDBParser()
ppb = Polypeptide.PPBuilder()

#test:
for index, row in skempi.iterrows():
  # Import PDB
  pdb=row['#Pdb'].split('_')[0]
  structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
  unique_pdb.add(pdb)
  
  #Progress
  if index%100==0: print(f'err:{len(err)} total:{len(unique_pdb)}')
    
  # Parse PDB to sequences
  chain_pp = ''.join([str(c.get_sequence()) for c in ppb.build_peptides(structure)])
  for mut in row['Mutation(s)_cleaned'].split(','):  # initAA/Chain/Index/mutAA
    initAA, chain, loc, mutAA = re.findall('(\d+|.)', mut)
    if mut[0] != chain_pp[int(loc)-1]: #if initAA != index of given chain
      err[pdb] = err[pdb]+1 if pdb in err.keys() else 1
      print(mut[0], chain_pp[int(loc)-2:int(loc)+3], mut)
  if index==100:
    break

print(err)

err:0 total:1
L ASHPD LI38G
L ASHPD LI38S
L ASHPD LI38P
L ASHPD LI38I
L ASHPD LI38D
L ASHPD LI38E
L FHFCG LI38G
L FHFCG LI38S
L FHFCG LI38P
L FHFCG LI38I
L FHFCG LI38D
L FHFCG LI38E
R SSHPD RI38K
K AGGAS KI46R
Y DLKVA YI42A
Y DLKVA YI42G
R AGGAS RI46A
R GASMV RI48A
R GASMV RI48C
R GASMV RI48A
R AGGAS RI46A
T SHPDL TI39D
T SHPDL TI39A
E PDLKV EI41A
T SHPDL TI39A
E PDLKV EI41A
T SHPDL TI39D
E PDLKV EI41A
A SHPDL AI39T
P SHPDL PI39T
A PDLKV AI41E
S PDLKV SI41E
S HPDLK SI40E
R SHPDL RI39M
A LKVAG AI43R
A AGGAS AI46R
T SHPDL TI39A
T SHPDL TI39P
E PDLKV EI41A
E PDLKV EI41S
E PDLKV EI41S
M HPDLK MI40R
R LKVAG RI43A
R AGGAS RI46A
R GASMV RI48A
G HPDLK GI40M
A HPDLK AI40M
K HPDLK KI40M
Y HPDLK YI40M
F HPDLK FI40M
A DLKVA AI42Y
M HPDLK MI40G
M HPDLK MI40A
M HPDLK MI40K
M HPDLK MI40Y
M HPDLK MI40F
Y DLKVA YI42A
M THVAG MI67E
M THVAG MI67D
M THVAG MI67H
M THVAG MI67K
M THVAG MI67R
M THVAG MI67G
M THVAG MI67A
M THVAG MI67L
M THVAG MI67V
M THVAG MI67I
err:19 total:21
{'1CSE': 6, '1ACB': 6, '1SBN': 1

#scrap work

In [0]:
parser = PDBParser() #PERMISSIVE=1
ppb = Polypeptide.PPBuilder()

pdb='3QIB'
structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')

for i in structure[0]:
  #print(list(i.get_residues()))
  #print(i.has_id(1))
  print(i)
  print(i.__getitem__(1)) #index starts @1
  print(i[1])
  print(ppb.build_peptides(i)[0].get_sequence())

In [0]:
print(list(structure.get_chains()))
print(list(structure.get_residues()))
print(structure[0]['B'])
print(structure[0]['B'][2])

print([c.id for c in structure.get_chains()])
print(list(map(lambda c:c.id, structure.get_chains())))

chain_ids = [c.id for c in structure.get_chains()]
chain_pp = ppb.build_peptides(structure)
dict(zip(chain_ids, chain_pp))

[<Chain id=A>, <Chain id=B>, <Chain id=P>, <Chain id=C>, <Chain id=D>]
[<Residue ILE het=  resseq=1 icode= >, <Residue LYS het=  resseq=2 icode= >, <Residue GLU het=  resseq=3 icode= >, <Residue GLU het=  resseq=4 icode= >, <Residue HIS het=  resseq=5 icode= >, <Residue THR het=  resseq=6 icode= >, <Residue ILE het=  resseq=7 icode= >, <Residue ILE het=  resseq=8 icode= >, <Residue GLN het=  resseq=9 icode= >, <Residue ALA het=  resseq=10 icode= >, <Residue GLU het=  resseq=11 icode= >, <Residue PHE het=  resseq=12 icode= >, <Residue TYR het=  resseq=13 icode= >, <Residue LEU het=  resseq=14 icode= >, <Residue LEU het=  resseq=15 icode= >, <Residue PRO het=  resseq=16 icode= >, <Residue ASP het=  resseq=17 icode= >, <Residue LYS het=  resseq=18 icode= >, <Residue ARG het=  resseq=19 icode= >, <Residue GLY het=  resseq=20 icode= >, <Residue GLU het=  resseq=21 icode= >, <Residue PHE het=  resseq=22 icode= >, <Residue MET het=  resseq=23 icode= >, <Residue PHE het=  resseq=24 icode= >, <

In [0]:
parser = PDBParser() #PERMISSIVE=1
ppb = Polypeptide.PPBuilder()

pdb='3QIB'
structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')

pp=ppb.build_peptides(structure)
print(pp[1].get_sequence())

SRPWFLEYCKSECHFYNGTQRVRLLVRYFYNLEENLRFDSDVGEFRAVTELGRPDAENWNSQPEFLEQKRAEVDTVCRHNYEIFDNFLVPRRVEPTVTVYP


In [0]:
parser = PDBParser() #PERMISSIVE=1
ppb = Polypeptide.PPBuilder()

pdb='1ACB'
structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
print(list(structure.get_chains()))
pp=ppb.build_peptides(structure)
print(str(pp[1].get_sequence())[36:41])
print('FLPE' in str(pp[1].get_sequence()))


[<Chain id=E>, <Chain id=I>]
VVTAA
False


In [0]:
parser = PDBParser(QUIET=True)  # PERMISSIVE=1
ppb = Polypeptide.PPBuilder(radius=1000)

pdb='1BRS'
structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
print(list(structure.get_chains()))
pp=ppb.build_peptides(structure)
print(str(pp[1].get_sequence()))

[<Chain id=A>, <Chain id=D>, <Chain id=B>, <Chain id=C>]
KKAVINGEQIRSISDLHQTLKKELALPEYYGENLDALWDALTGWVEYPLVLEWRQFEQSKQLTGAESVLQVFREAKAEGADITIILS


In [0]:
mut='AC1074K' #'EU208A'
#initAA, chain, loc, mutAA = re.findall('(\d+|.)', mut)
re.findall('(\d+|.)', mut)


['A', 'C', '1074', 'K']

#Still trouble getting mutation

In [0]:
errdct = {
'2BTF': 4, 
'1DAN': 3, 
'3BTE': 1, 
'3BTG': 1, 
'3BTH': 1, 
'3BTW': 1, 
'3D3V': 6, 
'4NKQ': 47,
'3R9A': 0
}


'''Mutations_cleaned V2'''
#import warnings
#warnings.filterwarnings("ignore")

#init:
err = dict()
unique_pdb = set()
parser = PDBParser(QUIET=True)  # PERMISSIVE=1
ppb = Polypeptide.PPBuilder(radius=1000)
pattern = '|'.join(errdct.keys())
skempi = skempi[skempi['#Pdb'].str.contains(pattern)]
#print(skempi)


#test:
for index, row in skempi.iterrows():
  # Import PDB
  pdb=row['#Pdb'].split('_')[0]
  structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
  unique_pdb.add(pdb)
  
  #Progress
  percent = 100*index/ln if index > 0 else 0
  if index%100==0: 
    print(f'{int(percent)}%', f'err:{len(err)} total:{len(unique_pdb)}')
    
  # Parse PDB to sequences
  chain_ids = [c.id for c in structure.get_chains()]
  chain_pp = [c.get_sequence() for c in ppb.build_peptides(structure)]
  Chain_seq = dict(zip(chain_ids, chain_pp))
  for mut in row['Mutation(s)_cleaned'].split(','):  # initAA/Chain/Index/mutAA
    initAA, chain, loc, mutAA = re.findall('(\d+|.)', mut)
    try:
      if initAA != Chain_seq[chain][int(loc)-1]: #if initAA != index of given chain
        err[pdb] = err[pdb]+1 if pdb in err.keys() else 1
        print(mut[0], Chain_seq[chain][int(loc)-2:int(loc)+3], mut, row['#Pdb'])
        print(Chain_seq)
    except:
      print(index, pdb, mut)
  #if index==100:
  #  break

print(err)

F AMYVA FP59A 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}
V MYVAI VP60E 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}
G ILTER GP120F 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}
K GYSFT KP125A 2BTF_A_P
{'A': Seq('DDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVG...PIE', ProteinAlphabet()), 'P': Seq('GIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETF...KCF', ProteinAlphabet())}
2043 1DAN WT40R
2043 1DAN ST42G
2044 1DAN WT40A
2045 1DAN ST42A
2046 1DAN VT28A
2047 1DAN YT46A
2048 1DAN FT71A
2049 1DAN YT73A
P LRPGS PU2A 1DAN_HL_UT
{'H': 

In [1]:
parser = PDBParser(QUIET=True)  # PERMISSIVE=1
ppb = Polypeptide.PPBuilder()#radius=100000)

pdb='2BTF'
structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
print(list(structure.get_chains()))
pp=ppb.build_peptides(structure)
print(str(pp[1].get_sequence())[120])

#1DAN VT28A

NameError: ignored

In [6]:
parser = PDBParser(QUIET=True)  # PERMISSIVE=1
ppb = Polypeptide.PPBuilder()#radius=100000)

pdb='2BTF'
structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
print(list(structure.get_chains()))
pp=ppb.build_peptides(structure)
print(str(pp[1].get_sequence())[120])

print(structure[0]['P'][59].resname)
#print(pp[1][59].resname)
print(pp[1])

[<Chain id=A>, <Chain id=P>]
T
PHE
<Polypeptide start=73 end=374>


In [7]:
print(list(structure.get_chains()))
print(list(structure.get_residues()))

for chain in structure.get_chains():
  print(chain.id, [c.resname for c in chain.get_residues()])

[<Chain id=A>, <Chain id=P>]
[<Residue ASP het=  resseq=1 icode= >, <Residue ASP het=  resseq=2 icode= >, <Residue ASP het=  resseq=3 icode= >, <Residue ILE het=  resseq=4 icode= >, <Residue ALA het=  resseq=5 icode= >, <Residue ALA het=  resseq=6 icode= >, <Residue LEU het=  resseq=7 icode= >, <Residue VAL het=  resseq=8 icode= >, <Residue VAL het=  resseq=9 icode= >, <Residue ASP het=  resseq=10 icode= >, <Residue ASN het=  resseq=11 icode= >, <Residue GLY het=  resseq=12 icode= >, <Residue SER het=  resseq=13 icode= >, <Residue GLY het=  resseq=14 icode= >, <Residue MET het=  resseq=15 icode= >, <Residue CYS het=  resseq=16 icode= >, <Residue LYS het=  resseq=17 icode= >, <Residue ALA het=  resseq=18 icode= >, <Residue GLY het=  resseq=19 icode= >, <Residue PHE het=  resseq=20 icode= >, <Residue ALA het=  resseq=21 icode= >, <Residue GLY het=  resseq=22 icode= >, <Residue ASP het=  resseq=23 icode= >, <Residue ASP het=  resseq=24 icode= >, <Residue ALA het=  resseq=25 icode= >, <Res

##Functional

Sang Young Noh of the Optimus Bind Project used an alternative method, converting AA's to match rather than converting to sequence. It is a bit disconcerting that PPBuilder does not operate as intended. This may point to underlying issues in the PDB's or bug.

In [26]:
from Bio.PDB.Polypeptide import aa1 #  aa1 = 'ACDEFGHIKLMNPQRSTVWY'
from Bio.PDB.Polypeptide import aa3

AA1to3 = dict(zip(aa1, aa3))

errdct = {
'2BTF': 4, 
'1DAN': 3, 
'3BTE': 1, 
'3BTG': 1, 
'3BTH': 1, 
'3BTW': 1, 
'3D3V': 6, 
'4NKQ': 47,
'3R9A': 0
}


'''Mutations_cleaned V2'''
#import warnings
#warnings.filterwarnings("ignore")

#init:
err = dict()
unique_pdb = set()
parser = PDBParser(QUIET=True)  # PERMISSIVE=1
pattern = '|'.join(errdct.keys())
ln = len(skempi)#.shape[0]
skempi2 = skempi[skempi['#Pdb'].str.contains(pattern)]
#print(skempi)


#test:
for index, row in skempi2.iterrows():
  # Import PDB
  pdb=row['#Pdb'].split('_')[0]
  structure = parser.get_structure(pdb, f'PDBs/{pdb}.pdb')
  unique_pdb.add(pdb)
  
  #Progress
  percent = 100*index/ln if index > 0 else 0
  if index%100==0: 
    print(f'{int(percent)}%', f'err:{len(err)} total:{len(unique_pdb)}')
    
  # Parse PDB to residues list
  Chain_seq = dict()
  for chain in structure.get_chains():
    Chain_seq[chain.id] = [c.resname for c in chain.get_residues()]
    
  # Mutation Comparator
  for mut in row['Mutation(s)_cleaned'].split(','):  # initAA/Chain/Index/mutAA
    initAA, chain, loc, mutAA = re.findall('(\d+|.)', mut)
    try:
      if AA1to3[initAA] != Chain_seq[chain][int(loc)-1]: #if initAA != index of given chain
        err[pdb] = err[pdb]+1 if pdb in err.keys() else 1
        print(mut[0], Chain_seq[chain][int(loc)-2:int(loc)+3], mut, row['#Pdb'])
        print(Chain_seq)
    except:
      print(index, pdb, mut)
  if index==500:
    break

print(err)

29% err:0 total:2
43% err:0 total:2
88% err:0 total:9
{}
